**Portfolio Project 2**

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a_b_test_results = '/content/drive/MyDrive/MATE/a_b_t.csv'
df = pd.read_csv(a_b_test_results)
df

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Jordan,desktop,Asia,Organic Search,2,1,session with orders,1
1,2020-11-03,Qatar,desktop,Asia,Organic Search,2,1,session with orders,1
2,2020-11-03,Slovakia,desktop,Europe,Organic Search,2,2,session with orders,1
3,2020-11-03,New Zealand,mobile,Oceania,Direct,2,1,session with orders,1
4,2020-11-03,Venezuela,mobile,Americas,Paid Search,2,1,session with orders,1
...,...,...,...,...,...,...,...,...,...
800991,2020-12-13,Vietnam,desktop,Asia,Direct,3,2,view_promotion,1
800992,2020-12-23,Vietnam,mobile,Asia,Paid Search,4,1,page_view,1
800993,2020-11-07,Vietnam,mobile,Asia,Paid Search,2,2,session_start,1
800994,2020-12-08,Vietnam,mobile,Asia,Direct,3,1,view_promotion,1


In [ ]:
df['event_name'] = df['event_name'].replace('account', 'new_account')
event_counts = df['event_name'].value_counts()  # Підраховуємо частоту оновлених значень
print(event_counts)

event_name
session                107210
session_start          106242
page_view              101907
user_engagement         94520
first_visit             81621
scroll                  73643
view_promotion          61695
view_item               44869
session with orders     25892
new account             22389
view_search_results     14310
add_shipping_info       11961
begin_checkout          11959
select_item             11816
add_to_cart             11762
select_promotion         9044
add_payment_info         8048
click                    1997
view_item_list            111
Name: count, dtype: int64


In [ ]:
import pandas as pd
import statsmodels.api as sm

metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']
# Створюємо пустий датафрейм для наповнення його кінцевими розрахунками
df_result = pd.DataFrame()

# 2. Цикли по тестах та метриках
for number_of_test in (1, 2, 3, 4):   # Цикл по тестах
    for m in metrics:              # Вкладений цикл, щоб пройти по всім метрикам
        try:
            # 3. Фільтрація та створення зведеної таблиці
            # Робимо зріз початкового датафрейму по конкретному тесту
            df1 = df[df['test'] == number_of_test]

            # Перевіряємо, чи є взагалі дані для цього тесту, перш ніж йти далі
            if df1.empty:
                print(f"Інформація: Для тесту {number_of_test} немає даних. Пропускаємо.")
                break # Якщо для тесту немає даних, переходимо до наступного номеру тесту

            # Формуємо зведену таблицю. fill_value=0 замінює можливі пропуски нулями, що запобігає помилкам.
            pivot_table = pd.pivot_table(df1, values='value', index='event_name', columns='test_group', aggfunc="sum", fill_value=0)

            # 4. Перевірка наявності даних перед розрахунком
            # Перевіряємо, чи існують у тесті групи А (1) та B (2)
            if 1 not in pivot_table.columns or 2 not in pivot_table.columns:
                print(f"Інформація: У тесті {number_of_test} відсутні тестові групи 1 та/або 2. Пропускаємо метрику '{m}'.")
                continue # Переходимо до наступної метрики

            # 5. Розрахунок змінних (безпечний доступ до даних)
            # Використовуємо виправлену назву події 'session'
            numerator_converse_A = pivot_table.loc[m, 1]
            denominator_converse_A = pivot_table.loc["session", 1]

            numerator_converse_B = pivot_table.loc[m, 2]
            denominator_converse_B = pivot_table.loc["session", 2]

            # Перевірка на ділення на нуль
            if denominator_converse_A == 0 or denominator_converse_B == 0:
                print(f"Інформація: У тесті {number_of_test} для метрики '{m}' відсутні сесії в одній із груп. Пропускаємо.")
                continue

            # 6. Розрахунок конверсій та їх зміни
            conversion_rate_A = numerator_converse_A / denominator_converse_A
            conversion_rate_B = numerator_converse_B / denominator_converse_B

            # Обробка випадку, коли конверсія в групі А дорівнює 0, щоб уникнути ділення на нуль
            if conversion_rate_A == 0:
                metric_change = float('inf') if conversion_rate_B > 0 else 0
            else:
                metric_change = ((conversion_rate_B / conversion_rate_A) - 1) * 100

            # 7. Розрахунок статистичної значущості (Z-тест)
            # ---- ВИПРАВЛЕННЯ ----
            # Міняємо місцями дані груп, щоб розрахунок був (Тестова група B - Контрольна група A)
            z_stat, p_value = sm.stats.proportions_ztest(
                [numerator_converse_B, numerator_converse_A],
                [denominator_converse_B, denominator_converse_A]
            )
            # ---- КІНЕЦЬ ВИПРАВЛЕННЯ ----


            # 8. Створення та наповнення фінального DataFrame
            df_test = pd.DataFrame({
                                    'test_number': number_of_test,
                                    'metric': f"{m}/session",
                                    'numerator_event': m,
                                    'denominator_event': 'session',
                                    'numerator_converse_A': numerator_converse_A,
                                    'denominator_converse_A' : denominator_converse_A,
                                    'conversion_rate_A': conversion_rate_A,
                                    'numerator_converse_B': numerator_converse_B,
                                    'denominator_converse_B': denominator_converse_B,
                                    'conversion_rate_B': conversion_rate_B,
                                    'metric_change': metric_change,
                                    'z_stat': z_stat,
                                    'p_value': p_value,
                                    'significant': p_value < 0.05
                                    }, index=[0])

            # Додаємо до початкового пустого датафрейму датафрейм з порахованими метриками
            df_result = pd.concat([df_result, df_test], ignore_index=True)

        # 9. Обробка помилки, якщо подія взагалі не знайдена
        except KeyError as e:
            print(f"Попередження: У тесті {number_of_test} не знайдено подію {e}. Перевірте, чи правильні назви метрик та події 'session'.")
            continue

# --- Кінець коду ---

# Вивід фінального результату
print("\n--- Фінальний результат аналізу (з виправленою z-статистикою) ---")
print(df_result)


--- Фінальний результат аналізу (з виправленою z-статистикою) ---
    test_number                     metric    numerator_event  \
0             1   add_payment_info/session   add_payment_info   
1             1  add_shipping_info/session  add_shipping_info   
2             1     begin_checkout/session     begin_checkout   
3             1        new account/session        new account   
4             2   add_payment_info/session   add_payment_info   
5             2  add_shipping_info/session  add_shipping_info   
6             2     begin_checkout/session     begin_checkout   
7             2        new account/session        new account   
8             3   add_payment_info/session   add_payment_info   
9             3  add_shipping_info/session  add_shipping_info   
10            3     begin_checkout/session     begin_checkout   
11            3        new account/session        new account   
12            4   add_payment_info/session   add_payment_info   
13            4  add_sh

In [ ]:
df_result

,test_number,metric,numerator_event,denominator_event,numerator_converse_A,denominator_converse_A,conversion_rate_A,numerator_converse_B,denominator_converse_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,1,add_payment_info/session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,3.924884,0.000087,True
1,1,add_shipping_info/session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,2.603571,0.009226,True
2,1,begin_checkout/session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,2.978783,0.002894,True
3,1,new account/session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,-1.542883,0.122859,False
4,2,add_payment_info/session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,1.240994,0.214608,False
5,2,add_shipping_info/session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,0.709557,0.477979,False
6,2,begin_checkout/session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,0.952898,0.340642,False
7,2,new account/session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,0.588793,0.556000,False
8,3,add_payment_info/session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,0.643172,0.520112,False
9,3,add_shipping_info/session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,-1.413727,0.157442,False


**Фінальний аналіз результатів:**

    Тест 1: Успішний
        Що сталося: Статистично значуще зростання конверсії на всіх ключових етапах воронки: додавання платіжної інформації (+12.5%), інформації про доставку (+6.6%) та початку оформлення замовлення (+6.7%).
        Висновок: Зміни, впроваджені в цьому тесті, є ефективними. Їх варто розгортати на всіх користувачів.

    Тест 2: Нейтральний
        Що сталося: Жодних статистично значущих змін не виявлено. Усі p-value > 0.05.
        Висновок: Зміни не мали ані позитивного, ані негативного впливу. Впроваджувати їх немає сенсу.

    Тест 3: Невдалий
        Що сталося: Статистично значуще падіння конверсії в початок оформлення замовлення (begin_checkout) на 3.4%.
        Висновок: Зміни шкідливі для ключової метрики. Їх слід відхилити.

    Тест 4: Вкрай невдалий
        Що сталося: Статистично значуще падіння конверсії одразу за двома метриками: початок оформлення замовлення (begin_checkout) на 2.4% та створення нового акаунта (new account) на 3.4%.
        Висновок: Зміни негативно впливають на поведінку користувачів. Їх однозначно слід відхилити.

**Фінальний аналіз результатів A/B Тестування**

Було проведено аналіз чотирьох тестів, спрямованих на покращення користувацького досвіду та конверсії. Нижче наведено детальні висновки по кожному тесту та загальні рекомендації.

*Загальні висновки та Рекомендації:*

Тест	Результат	Ключові зміни (статистично значущі)
	Рішення
Тест 1.	Успішний	+12.5%. Додавання платіжної інфо >+ 6.7%. Початок оформлення замовлення >+ 6.6%. Додавання інфо про доставку.	Впровадити.

Тест 2.	Нейтральний.	Жодних значущих змін.	Відхилити.

Тест 3.	Невдалий	-3.4%. Початок оформлення замовлення.	Відхилити.

Тест 4.	Вкрай невдалий	-2.4%. Початок оформлення замовлення > -3.4%. Створення нового акаунта.	Відхилити.

**Головна рекомендація:** Необхідно негайно розпочати процес повноцінного розгортання змін з Тесту 1 на всіх користувачів. Водночас слід повністю відхилити зміни, перевірені в Тестах 2, 3 та 4, оскільки вони не показали позитивного ефекту або завдали прямої шкоди ключовим метрикам.
Детальний аналіз по кожному тесту
Тест 1: Успішний ✅

Зміни, впроваджені в цьому тесті, продемонстрували значне та статистично підтверджене покращення на ключових етапах воронки конверсії.

    Що сталося:
        Конверсія в додавання платіжної інформації (add_payment_info) зросла на 12.5% (ptext−value=0.000087).
        Конверсія в додавання інформації про доставку (add_shipping_info) зросла на 6.6% (ptext−value=0.009226).
        Конверсія в початок оформлення замовлення (begin_checkout) зросла на 6.7% (ptext−value=0.002894).
    Висновок: Цей тест є однозначно успішним. Позитивний вплив спостерігається на кількох важливих метриках, що свідчить про високу ефективність внесених змін.
    Рішення: Розгортати на 100% аудиторії.

Тест 2: Нейтральний 🟡

Зміни не мали жодного статистично значущого впливу на поведінку користувачів.

    Що сталося: Жодна з метрик, що відстежувалися (включно з add_payment_info, add_shipping_info, begin_checkout, new_account), не показала значущих змін. Усі показники ptext−value перевищують поріг 0.05.
    Висновок: Зміни виявилися нейтральними. Вони не покращили, але й не погіршили ключові показники.
    Рішення: Відхилити. Впровадження не має сенсу через відсутність позитивного ефекту.

Тест 3: Невдалий ❌

Цей варіант змін негативно вплинув на один з найважливіших етапів воронки — початок оформлення замовлення.

    Що сталося: Зафіксовано статистично значуще падіння конверсії в початок оформлення замовлення (begin_checkout) на 3.4% (ptext−value=0.012026).
    Висновок: Зміни є шкідливими для бізнесу, оскільки вони знижують бажання користувачів розпочинати процес покупки.
    Рішення: Відхилити.

Тест 4: Вкрай невдалий ⛔

Зміни з цього тесту завдали шкоди одразу за двома важливими напрямками, що свідчить про їхній глибоко негативний вплив на користувацький досвід.

    Що сталося:
        Конверсія в початок оформлення замовлення (begin_checkout) впала на 2.4% (ptext−value=0.045934).
        Конверсія в створення нового акаунта (new_account) впала на 3.4% (ptext−value=0.017527).
    Висновок: Тест продемонстрував подвійний негативний ефект, погіршивши як метрики, пов'язані з конверсією, так і метрики залучення нових користувачів.
    Рішення: Однозначно відхилити.

In [ ]:
# Зберігаємо результати в CSV файл
df_result.to_csv('/content/drive/MyDrive/MATE/test_results_final.csv', index=False)

In [ ]:
df_result.to_excel('test_table.xlsx')

**Посилання на файл**

https://drive.google.com/file/d/1AepMZjUMi2SzFJHZX8J7zafrX__4ij-a/view?usp=sharing

**Посилання на Дашборд**

https://public.tableau.com/app/profile/oleksandr.oleksandr7187/viz/ABtest-Calculators-3/Significance